In [13]:
import faker
import mysql.connector  # Cambiamos psycopg2 por mysql.connector
from datetime import datetime
import random
import time

In [14]:
fake = faker.Faker()

def generate_transaction():
    user = fake.simple_profile()

    return {
        "transactionId": fake.uuid4(),
        "userId": user['username'],
        "timestamp": datetime.utcnow().timestamp(),
        "amount": round(random.uniform(10, 1000), 2),        
        'city': fake.city(),
        "country": fake.country(),
        "merchantName": fake.company(),
        "paymentMethod": random.choice(['credit_card', 'debit_card', 'online_transfer']),        
        'affiliateId': fake.uuid4()
    }

def create_table(conn):
    cursor = conn.cursor()

    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS transactions (
            transaction_id VARCHAR(255) PRIMARY KEY,
            user_id VARCHAR(255),
            timestamp TIMESTAMP,
            amount DECIMAL(10,2),
            city VARCHAR(255),
            country VARCHAR(255),
            merchant_name VARCHAR(255),
            payment_method VARCHAR(255),       
            affiliateId VARCHAR(255)
        )
        """)

    cursor.close()
    conn.commit()


In [22]:
if __name__ == "__main__":
    conn = mysql.connector.connect(
        host='mysql',  # Cambiamos 'postgres' por 'mysql'
        database='test_binlog',
        user='root',  # Usuario común en MySQL
        password='root',  # Contraseña común en MySQL
        port=3306  # Puerto por defecto de MySQL
    )

    create_table(conn)

    transaction = generate_transaction()
    cur = conn.cursor()
    print(transaction)
    
    cur.execute(
        """
        INSERT INTO transactions(transaction_id, user_id, timestamp, amount, city, country, merchant_name, payment_method, affiliateId)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """, (transaction["transactionId"], transaction["userId"], datetime.fromtimestamp(transaction["timestamp"]).strftime('%Y-%m-%d %H:%M:%S'),
              transaction["amount"], transaction["city"], transaction["country"],
              transaction["merchantName"], transaction["paymentMethod"], transaction["affiliateId"])
    )        

    cur.close()
    conn.commit()

{'transactionId': '07b373f2-3d8f-4e9a-a00b-f2e1b849054f', 'userId': 'williamsdebra', 'timestamp': 1748319517.974294, 'amount': 660.42, 'city': 'South Robert', 'country': 'Moldova', 'merchantName': 'Morgan and Sons', 'paymentMethod': 'debit_card', 'affiliateId': '14b6a579-da35-44d7-879b-51319e0e76f7'}
